**Primera iteración de Resolución del Problema**

**Tarea 1: Clasificación de Toxicidad**

Recibir una frase y detectar si es tóxica o no

In [1]:
import pandas as pd

In [2]:
# Cargar tus datos (adapta los nombres de columna)
english_paradetox_df = pd.read_csv("datos/english_paradetox_preprocessed.csv")

# Crear dataframes separados
df_toxic = pd.DataFrame({
    'text': english_paradetox_df['toxic_sentence_tokens'],
    'label': 1
})

df_neutral = pd.DataFrame({
    'text': english_paradetox_df['neutral_sentence_tokens'],
    'label': 0
})

# Combinar y barajar
df_clasificacion = pd.concat([df_toxic, df_neutral]).sample(frac=1, random_state=42).reset_index(drop=True)

# Limpiar textos nulos si los hubiera
df_clasificacion = df_clasificacion.dropna(subset=['text'])

In [3]:
from sklearn.model_selection import train_test_split

X = df_clasificacion['text']
y = df_clasificacion['label']

# 80% para entrenar, 20% para testear
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [4]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report

# Crear el pipeline
pipeline_nb = Pipeline([
    ('vectorizer_bow', CountVectorizer()),  # 1. Vectorizar (BoW)
    ('classifier_nb', MultinomialNB())       # 2. Clasificar (Naive Bayes)
])

# Entrenar
pipeline_nb.fit(X_train, y_train)

# Predecir y evaluar
y_pred_nb = pipeline_nb.predict(X_test)
print("--- Resultados Naive Bayes + BoW ---")
print(classification_report(y_test, y_pred_nb))

# Guarda estos resultados para tu tabla final

--- Resultados Naive Bayes + BoW ---
              precision    recall  f1-score   support

           0       0.60      0.41      0.49        80
           1       0.55      0.72      0.63        80

    accuracy                           0.57       160
   macro avg       0.58      0.57      0.56       160
weighted avg       0.58      0.57      0.56       160



In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC 
# (o usa LogisticRegression, ambos son válidos y fuertes)
from sklearn.linear_model import LogisticRegression 

# Crear el pipeline
pipeline_svm = Pipeline([
    ('vectorizer_tfidf', TfidfVectorizer()),  # 1. Vectorizar (TF-IDF)
    ('classifier_svm', SVC(kernel='linear'))  # 2. Clasificar (SVM Lineal)
    # (Alternativa: 'classifier_lr', LogisticRegression(max_iter=1000))
])

# Entrenar
pipeline_svm.fit(X_train, y_train)

# Predecir y evaluar
y_pred_svm = pipeline_svm.predict(X_test)
print("--- Resultados SVM + TF-IDF ---")
print(classification_report(y_test, y_pred_svm))

# Guarda estos resultados

--- Resultados SVM + TF-IDF ---
              precision    recall  f1-score   support

           0       0.72      0.81      0.76        80
           1       0.79      0.69      0.73        80

    accuracy                           0.75       160
   macro avg       0.75      0.75      0.75       160
weighted avg       0.75      0.75      0.75       160



In [6]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

# Configurar el tokenizador de Keras
vocab_size = 2000  # Número máximo de palabras a retener
max_length = 50    # Longitud máxima de las secuencias (tweets)
trunc_type = 'post'
padding_type = 'post'

tokenizer = Tokenizer(num_words=vocab_size, oov_token="<OOV>")
tokenizer.fit_on_texts(X_train) # ¡Importante! Ajustar SOLO con datos de train

# Convertir textos a secuencias
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

# Aplicar padding"""  """
X_train_pad = pad_sequences(X_train_seq, maxlen=max_length, padding=padding_type, truncating=trunc_type)
X_test_pad = pad_sequences(X_test_seq, maxlen=max_length, padding=padding_type, truncating=trunc_type)

# Convertir etiquetas a numpy arrays (necesario para Keras)
y_train_np = np.array(y_train)
y_test_np = np.array(y_test)

In [7]:
import numpy as np

# Define la dimensión de los embeddings
embedding_dim = 64

# Ruta al archivo GloVe
glove_file_path = 'glove100b.txt' 

# 1. Cargar el modelo GloVe en un diccionario
embeddings_index = {}
with open(glove_file_path, 'r', encoding='utf-8') as f:
    for line in f:
        values = line.split()
        
        # Primero, una comprobación básica:
        # Si la línea está en blanco o es demasiado corta, la saltamos.
        if len(values) < 2:
            continue
            
        word = values[0]
        
        # --- ESTA ES LA CORRECCIÓN ---
        try:
            # Intenta convertir el resto en números
            coefs = np.asarray(values[1:], dtype='float32')
            
            # Comprobación extra: ¿tiene la dimensión correcta?
            if len(coefs) == embedding_dim:
                embeddings_index[word] = coefs
            # else:
                # Opcional: avisar si una palabra tiene dimensiones raras
                # print(f"Palabra '{word}' saltada: dimensión incorrecta ({len(coefs)})")

        except ValueError:
            # Si falla la conversión (ej. encuentra un '.'), 
            # simplemente ignora esta línea y pasa a la siguiente.
            # print(f"Línea malformada saltada: {line[:30]}...") # (Descomenta para depurar)
            continue
        # --- FIN DE LA CORRECCIÓN ---

print(f"Se encontraron {len(embeddings_index)} vectores de palabras en GloVe.")

Se encontraron 0 vectores de palabras en GloVe.


In [8]:
# 2. Obtener el word_index de tu tokenizer
# (Asumiendo que 'tokenizer' es tu objeto Tokenizer ya ajustado)
word_index = tokenizer.word_index

# 3. Crear la matriz de embeddings (vocab_size x embedding_dim)
# (Asumiendo que 'vocab_size' es el que definiste antes, ej: 10000)
embedding_matrix = np.zeros((vocab_size, embedding_dim))

# 4. Rellenar la matriz
# Iteramos sobre cada palabra en NUESTRO vocabulario (el del tokenizer)
for word, i in word_index.items():
    if i >= vocab_size:
        continue  # No incluir palabras más allá del vocab_size
        
    # Buscamos el vector de la palabra en el índice de GloVe
    embedding_vector = embeddings_index.get(word)
    
    if embedding_vector is not None:
        # ¡Palabra encontrada! La ponemos en la fila correcta (índice 'i')
        embedding_matrix[i] = embedding_vector
    # Si 'embedding_vector' es None, la palabra no estaba en GloVe.
    # Se quedará como un vector de ceros en la matriz (la capa Embedding aprenderá su vector desde cero).

In [9]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense

# --- Experimento "CONGELADO" ---
model_cnn_frozen = Sequential([
    Embedding(
        input_dim=vocab_size,       # Altura de la matriz (ej. 10000)
        output_dim=embedding_dim,   # Anchura de la matriz (ej. 100)
        input_length=max_length,
        weights=[embedding_matrix], # ¡Aquí la cargas!
        trainable=False             # ¡CONGELADO! No se actualiza en el entreno
    ),
    Conv1D(128, 5, activation='relu'),
    GlobalMaxPooling1D(),
    Dense(1, activation='sigmoid')
])

model_cnn_frozen.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# --- Experimento "FINE-TUNEADO" ---
model_cnn_finetune = Sequential([
    Embedding(
        input_dim=vocab_size, 
        output_dim=embedding_dim,
        input_length=max_length,
        weights=[embedding_matrix], # ¡Aquí la cargas!
        trainable=True              # ¡FINE-TUNEADO! Se actualiza en el entreno
    ),
    Conv1D(128, 5, activation='relu'),
    GlobalMaxPooling1D(),
    Dense(1, activation='sigmoid')
])

model_cnn_finetune.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

c:\Users\js834\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [10]:
# --- ENTRENAR EL MODELO "CONGELADO" ---
print("--- Entrenando Modelo CONGELADO (GloVe) ---")
history_frozen = model_cnn_frozen.fit(
    X_train_pad,          # Tus frases de entrenamiento (con padding)
    y_train_np,           # Tus etiquetas de entrenamiento
    epochs=100,             # Número de vueltas (puedes empezar con 5)
    batch_size=64,
    validation_data=(X_test_pad, y_test_np), # ¡Importante! Usa los datos de test para validar
    verbose=1
)

# --- ENTRENAR EL MODELO "FINE-TUNEADO" ---
print("\n--- Entrenando Modelo FINE-TUNEADO (GloVe) ---")
history_finetune = model_cnn_finetune.fit(
    X_train_pad,
    y_train_np,
    epochs=100,
    batch_size=64,
    validation_data=(X_test_pad, y_test_np),
    verbose=1
)

--- Entrenando Modelo CONGELADO (GloVe) ---
Epoch 1/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - accuracy: 0.4938 - loss: 0.6932 - val_accuracy: 0.5000 - val_loss: 0.6931
Epoch 2/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.5000 - loss: 0.6932 - val_accuracy: 0.5000 - val_loss: 0.6931
Epoch 3/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.5000 - loss: 0.6931 - val_accuracy: 0.5000 - val_loss: 0.6931
Epoch 4/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.5000 - loss: 0.6932 - val_accuracy: 0.5000 - val_loss: 0.6931
Epoch 5/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.5000 - loss: 0.6931 - val_accuracy: 0.5000 - val_loss: 0.6931
Epoch 6/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.5000 - loss: 0.6932 - val_accuracy: 0.5000 - val_loss: 0.6931
Epoch 7/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.5000 - loss: 0.6931 - val_accuracy: 0.5000 - val_loss: 0.6931
Epoch 8/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.5

In [11]:
from sklearn.metrics import classification_report

# --- EVALUAR EL MODELO "CONGELADO" ---

# 1. Predecir sobre los datos de test
# .predict() te da probabilidades (ej. 0.87, 0.12)
y_pred_prob_frozen = model_cnn_frozen.predict(X_test_pad)

# 2. Convertir probabilidades a etiquetas (0 o 1)
# Si la probabilidad es > 0.5, decimos que es 1 (Tóxico)
y_pred_frozen = (y_pred_prob_frozen > 0.5).astype("int32")

# 3. Mostrar el reporte
print("\n--- Reporte de Clasificación: Modelo CONGELADO ---")
print(classification_report(y_test_np, y_pred_frozen))


# --- EVALUAR EL MODELO "FINE-TUNEADO" ---

# 1. Predecir
y_pred_prob_finetune = model_cnn_finetune.predict(X_test_pad)

# 2. Convertir
y_pred_finetune = (y_pred_prob_finetune > 0.5).astype("int32")

# 3. Mostrar el reporte
print("\n--- Reporte de Clasificación: Modelo FINE-TUNEADO ---")
print(classification_report(y_test_np, y_pred_finetune))

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 

--- Reporte de Clasificación: Modelo CONGELADO ---
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        80
           1       0.50      1.00      0.67        80

    accuracy                           0.50       160
   macro avg       0.25      0.50      0.33       160
weighted avg       0.25      0.50      0.33       160

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 

--- Reporte de Clasificación: Modelo FINE-TUNEADO ---
              precision    recall  f1-score   support

           0       0.50      1.00      0.67        80
           1       0.00      0.00      0.00        80

    accuracy                           0.50       160
   macro avg       0.25      0.50      0.33       160
weighted avg       0.25      0.50      0.33       160



c:\Users\js834\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\js834\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\js834\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo